In [ ]:
!pip install ultralytics opencv-python

## Run it Before running any of the Below Code

In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA Available: True
Using device: NVIDIA GeForce GTX 1650


#### CUDA donwload for 1650 gpu

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [9]:
# This is comment This is Basic First Code for Initializing Varibles 
#second line comment
import os
import cv2
import numpy as np
from ultralytics import YOLO
import threading
import time

# --- USER: VERIFY YOUR PROJECT DIRECTORY ---
PROJECT_DIR = "/content/drive/MyDrive/Colab_Projects/AI-Based-Intelligent-Traffic-Monitoring-and-Management-System"
# The name of your input video file, which should be in the 'data' subfolder
INPUT_VIDEO_FILENAME = "input.mp4"
# ---

# --- USER: DEFINE YOUR LANE'S ROI HERE ---
# Define the coordinates of YOUR lane's polygon
ROI_POINTS = np.array([(100, 300), (800, 300), (850, 450), (50, 450)], np.int32)
# ---

# Construct full paths for video files
input_video_path = os.path.join(PROJECT_DIR, 'data', INPUT_VIDEO_FILENAME)
output_video_path = os.path.join(PROJECT_DIR, 'output', 'output_vehicle_count.mp4')

# Ensure the output directory exists
os.makedirs(os.path.join(PROJECT_DIR, 'output'), exist_ok=True)

# Change the current working directory to the project folder
os.chdir(PROJECT_DIR)

# Load the pre-trained YOLOv8 model
print("Loading YOLOv8 model...")
model = YOLO("yolov8n.pt").to("cuda")


# COCO class IDs for vehicles (car, motorcycle, bus, truck)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

Loading YOLOv8 model...


# Video Processing especially for google Colab

In [ ]:
# Open the input video file
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print(f"Error: Could not open video at {input_video_path}")
else:
    # Get video properties for the output file
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    print(f"Processing video: {INPUT_VIDEO_FILENAME}...")

try:
    while True:
        success, frame = cap.read()
        if not success:
            break # End of video

        # This check prevents errors if a frame is corrupted
        if frame.shape[1] != frame_width or frame.shape[0] != frame_height:
            frame = cv2.resize(frame, (frame_width, frame_height))

        # Reset counter for each frame
        vehicle_count = 0

        # Draw the ROI polygon once per frame
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model(frame)

        # Process detections
        for box in results[0].boxes:
            cls_id = int(box.cls)
            if cls_id in VEHICLE_CLASS_IDS:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    label = model.names[cls_id]  # e.g., 'car', 'bus'
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    # Put label above the bounding box
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                0.8, (0, 255, 0), 2)

        # Write the final count and the frame to the output video
        cv2.putText(frame, f"Vehicles in Lane: {vehicle_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)

finally:
    # This ensures files are closed properly even if an error occurs
    print("Processing complete. Releasing resources.")
    cap.release()
    out.release()

# Accessing Through Web Cam on Local Pc "Live"

In [3]:


# --- Load YOLO model ---
model = YOLO("yolov8n.pt").to("cuda")

# --- ROI (adjust to your camera view) ---
ROI_POINTS = np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32)

# Vehicle class IDs (car=2, motorcycle=3, bus=5, truck=7)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# --- Open webcam ---
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # try without CAP_DSHOW if on Linux/Mac

if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

print("🚀 Starting live detection... Press 'q' to quit.")

try:
    while True:
        success, frame = cap.read()
        if not success:
            print("❌ Failed to grab frame")
            break

        # Reset vehicle counter
        vehicle_count = 0

        # Draw ROI
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model.predict(frame, device=0, verbose = False )  # 0 = first GPU

        for box in results[0].boxes:
            cls_id = int(box.cls)
            conf = float(box.conf[0])

            if cls_id in VEHICLE_CLASS_IDS and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    label = f"{model.names[cls_id]} {conf:.2f}"

                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display count
        cv2.putText(frame, f"Vehicles in ROI: {vehicle_count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show live feed
        cv2.imshow("YOLO Vehicle Detection - Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    print("🛑 Releasing resources...")
    cap.release()
    cv2.destroyAllWindows()


🚀 Starting live detection... Press 'q' to quit.
🛑 Releasing resources...


In [4]:
# Two webcams in Parallel 

# --- Load YOLO model ---
model = YOLO("yolov8n.pt").to("cuda")

# --- ROI (adjust to your camera views separately if needed) ---
ROI_POINTS = np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32)

# Vehicle class IDs (car=2, motorcycle=3, bus=5, truck=7)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# --- Webcam processing function ---
def process_webcam(cam_index, window_name):
    cap = cv2.VideoCapture(cam_index, cv2.CAP_DSHOW)  # Use CAP_DSHOW only on Windows

    if not cap.isOpened():
        print(f"❌ Error: Could not open webcam {cam_index}")
        return

    print(f"🚀 Starting detection on webcam {cam_index}... Press 'q' to quit.")

    while True:
        success, frame = cap.read()
        if not success:
            print(f"❌ Failed to grab frame from webcam {cam_index}")
            break

        # Reset vehicle counter
        vehicle_count = 0

        # Draw ROI
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model.predict(frame, device=0, verbose=False)

        for box in results[0].boxes:
            cls_id = int(box.cls)
            conf = float(box.conf[0])

            if cls_id in VEHICLE_CLASS_IDS and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    label = f"{model.names[cls_id]} {conf:.2f}"

                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display count
        cv2.putText(frame, f"Vehicles in ROI: {vehicle_count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show live feed
        cv2.imshow(window_name, frame)

        # Press 'q' to quit both threads
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print(f"🛑 Releasing resources for webcam {cam_index}...")
    cap.release()
    cv2.destroyWindow(window_name)


# --- Run both webcams in parallel threads ---
t1 = threading.Thread(target=process_webcam, args=(0, "Webcam 0"))
t2 = threading.Thread(target=process_webcam, args=(1, "Webcam 1"))
t3 = threading.Thread(target=process_webcam, args=(2, "Webcam 2"))


t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

cv2.destroyAllWindows()


🚀 Starting detection on webcam 0... Press 'q' to quit.🚀 Starting detection on webcam 2... Press 'q' to quit.

🚀 Starting detection on webcam 1... Press 'q' to quit.
🛑 Releasing resources for webcam 0...
🛑 Releasing resources for webcam 2...
🛑 Releasing resources for webcam 1...


## Code with 4 different ROI's

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import threading

# --- Load YOLOv8 model ---
print("Loading YOLOv8 model...")
model = YOLO("yolov8n.pt").to("cuda")

# --- Vehicle class IDs ---
VEHICLE_CLASS_IDS = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# --- Define ONE ROI per webcam ---
ROIS = {
    0: np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32),   # ROI for Webcam 0
    1: np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32), # ROI for Webcam 1
    2: np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32), # ROI for Webcam 2
    3: np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32) # ROI for Webcam 3
}

# --- Webcam processing function ---
def process_webcam(cam_index, window_name):
    cap = cv2.VideoCapture(cam_index, cv2.CAP_DSHOW)

    if not cap.isOpened():
        print(f"❌ Error: Could not open webcam {cam_index}")
        return

    print(f"🚀 Starting detection on webcam {cam_index}... Press 'q' to quit.")

    roi_polygon = ROIS[cam_index]  # Get specific ROI for this cam

    while True:
        success, frame = cap.read()
        if not success:
            print(f"❌ Failed to grab frame from webcam {cam_index}")
            break

        # Reset count
        vehicle_count = 0

        # Draw ROI
        cv2.polylines(frame, [roi_polygon], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model.predict(frame, device=0, verbose=False)

        for box in results[0].boxes:
            cls_id = int(box.cls)
            conf = float(box.conf[0])

            if cls_id in VEHICLE_CLASS_IDS and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                # Check if vehicle lies inside ROI
                if cv2.pointPolygonTest(roi_polygon, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    label = f"{model.names[cls_id]} {conf:.2f}"

                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Show vehicle count
        cv2.putText(frame, f"Vehicles: {vehicle_count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show live feed
        cv2.imshow(window_name, frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print(f"🛑 Releasing resources for webcam {cam_index}...")
    cap.release()
    cv2.destroyWindow(window_name)


# --- Run 4 webcams in parallel ---
threads = []
for i in range(4):
    t = threading.Thread(target=process_webcam, args=(i, f"Webcam {i}"))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

cv2.destroyAllWindows()


Loading YOLOv8 model...
🚀 Starting detection on webcam 1... Press 'q' to quit.
🚀 Starting detection on webcam 0... Press 'q' to quit.
🚀 Starting detection on webcam 3... Press 'q' to quit.
Ultralytics 8.3.200  Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
🚀 Starting detection on webcam 2... Press 'q' to quit.
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
🛑 Releasing resources for webcam 1...
🛑 Releasing resources for webcam 3...
🛑 Releasing resources for webcam 0...
🛑 Releasing resources for webcam 2...
